mean filtering, and median filtering

In [1]:
import cv2
import numpy as np


In [2]:
video_path = 'highway.mp4' 
cap = cv2.VideoCapture(video_path)

In [3]:
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

In [ ]:
cv2.namedWindow('Foreground mask', cv2.WINDOW_NORMAL)

In [ ]:
average_frame = np.float32(frame)

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    cv2.accumulateWeighted(frame, average_frame, 0.01)
    background_frame = cv2.convertScaleAbs(average_frame)
    foreground_mask = cv2.absdiff(frame, background_frame)
    gray_foreground = cv2.cvtColor(foreground_mask, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(gray_foreground, 50, 255, cv2.THRESH_BINARY)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)
    binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)

    segmented_frame = cv2.bitwise_and(frame, frame, mask=binary_mask)

    cv2.imshow('Foreground Detection', segmented_frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break


In [ ]:
cap.release()
cv2.destroyAllWindows()

Median Filtering

In [4]:
from collections import deque

In [5]:
history_length = 10  
threshold_value = 50 

In [6]:
frame_history = deque(maxlen=history_length)

In [7]:
cv2.namedWindow('Foreground Detection', cv2.WINDOW_NORMAL)

In [8]:
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    frame_history.append(frame)

    if len(frame_history) == history_length:
        frames_array = np.array(frame_history)

        median_frame = np.median(frames_array, axis=0).astype(dtype=np.uint8)

        foreground_mask = cv2.absdiff(frame, median_frame)

        gray_foreground = cv2.cvtColor(foreground_mask, cv2.COLOR_BGR2GRAY)

        _, binary_mask = cv2.threshold(gray_foreground, threshold_value, 255, cv2.THRESH_BINARY)

        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)
        binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)

        segmented_frame = cv2.bitwise_and(frame, frame, mask=binary_mask)

        cv2.imshow('Foreground Detection', segmented_frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

In [9]:
cap.release()
cv2.destroyAllWindows()